In [214]:
#Import the necessaray packages

import spacy as sp                     #To parse text and other NLP operations

import re                              #Regex operations  
from heapq import nlargest             #Selecting top items

import xml.etree.cElementTree as ET    #To read XML file

import gensim                          #For topic modelling
import gensim.corpora as corpora

In [32]:
#Load the Spacy model. 
nlp  = sp.load("en_core_web_lg")

In [278]:
#The list object 'Stories' contains the User Stories from the excel file
Stories = ['As a Buyer, I want to edit my Profile, So that I can make updates', 
           'As a Administrator, I want to modify User Profile, So that I can correct user information',
           'As a Administrator, I want to delete non-User Profile, So that I can understand the count of active users',
           'As a Buyer, I want to modify my shoping cart, So that I can choose items to order',
           'As a Seller, I want to update products, So that I can see the right description',
           'As a Buyer, I want to make online payment for items in my shopping cart, So that I can order my items',
           'As a Seller or owner, I want to add products to my account, So that I can make them aavailable for sale',
           'As a Customer, I want to make cash payment, So that I can order my items'
          ]

#The list object 'AcceptanceCriteria' contains the Acceptance Criteria for each User Story
AcceptanceCriteria = [
         [1,'Buyer clicks the modify button to make the date editable'],[1,'Fields - DOB,Address, Landline, Mobile are editable'],[1,'Existing logic on field lengths to stay'],
         [2,'Adminsitrator clicks the Modify button to make changes'],[2,'Fields - isActive, Role, PrivelegeAccess are modifyable'],[2,'Existing logic on field lengths to stay'],
         [3,'Adminsitrator clicks the Delete button to receive a pop-up'],[3,'Pop-up has a message confirming delete action and requests user input (yes/no)'],[3,'Clicking Yes will delete the record from the table and move it to archive'],[3,'Pop-up has a message confirming delete action and requests user input (yes/no)'],[3,'Clicking No will cancel the delete operation'],
         [4,'Buyer clicks the edit cart button'],[4,'Items in the cart are displayed along with their quantity and price'],[4,'Buyer adds, removes or delete items in the cart'], [4,'Cart automatically updates content'],
         [5, 'Products are shown in a grid'], [5, 'Seller clicks the product to be updated'],[5,'Seller can update the photo, description and price of the product'], [5,'Seller clicks update button to save changed data'],[5,'A message is shown on update of products'],
         [6,'Buyer can choose types of payment'],[6,'Valid online payment types are Card (Credit/Debit), Banks providing online facility, Vouchers'],[6,'Buyer can choose a payment type and click next button'],[6,'Depending on the type of payment type, the corresponding page loads'],
         [7, 'Seller clicks the add buton to see a page'],[7, 'Fields - Product name, Product description, Product Photos, Product Price, Product Discounts, Product quantity'],[7,'Seller add information on all fields and clicks Add'],[7,'Product added and message of successful addition given to seller'],
         [8,'Customer given options of cards - MasterCard or Visa'], [8,'Customer chooses the card and enters Card details - 16 digit card number, Card holder name and cvv number'],[8,'Code validates the card type based on details'],[8,'On successful validation, payment gateway contacted'],[8,'Payment gateway results parsed and the message shared with Customer']
        ]

Role_XML = r'C:\AGILE-NLP\RulesData.xml'
Output_File = r'C:\AGILE-NLP\User_Stories_Summary.txt'


#Call each function to parse User Stories. Each of these functions write their observations to the output file
Independent (Stories,Output_File )
Negiotiable   (Stories,Output_File )
Valuable    (Stories,Role_XML, Output_File )
Estimable   (Stories,Output_File )
Small       (Stories,Output_File )
Testable    (Stories,AcceptanceCriteria, Output_File )


In [254]:
#Indepedent

#User Stories need to be independent of each other

def Independent(Stories, Output_File):
    
    #Initialize Variables:
    lstSummary=[]
    Summary=''
    User_Role=''
    User_Action=[' ']
    bflag = True

    #Logic:
    for i, User_Story in enumerate(Stories):
        User_Story = ''.join(User_Story.split(',', maxsplit=2)[:2])
        doc = nlp(User_Story)
        bflag = True #This is set for each User Story
        
        for tokens in doc:
            #Get the Role.Look at how we specifically look for certain 'types'. 
            #This is facilitated by how the User Stories are written
            if (tokens.pos_ =='NOUN' or tokens.pos_ =='PROPN') and tokens.dep_=='pobj' and  bflag==True:
                User_Role = tokens 
                bflag = False #The 'For' loop should not enter this piece of code again. Hence, using the bool variable.
                
            #The Category to which the Action is related to. 
            #Note this uses the list 'User_Action' got in next 'If' statement
            #Due to nature of user Story, this clause will never fire first. If it fires, chances are, User Story is written incorretly
            if tokens.text == str(User_Action[0]):
                Category = [child  for child in tokens.children if child.dep_=='dobj' ] #Direct Object of the Verb
            
            #The Action performed by the role
            if tokens.dep_ == 'ROOT' and tokens.text != User_Action:
                User_Action = [child  for child in tokens.children if child.dep_=='xcomp'] #The action intended in the User Story
                
        Summary = [i+1, str(Category[0]).capitalize(), str(User_Action[0]).capitalize(), str(User_Role).capitalize()]
        lstSummary.append(Summary)

    #We need to get the unique categories in the User Story
    strUniqueCategories = set([Category[1] for Category in lstSummary])
    
    #Write data to file
    with open(Output_File,'w') as Observations:
        Observations.write('Independent Criteria Observations:'+'\n')
        Observations.write(''.ljust(20) + 'User Story #' + ''.ljust(22) + 'Task' + ''.ljust(25) + 'Task Owner' + '\n' )         
        for uniCat in strUniqueCategories: 
            Observations.write ('Category:'+ uniCat+'\n')
            lstChildren = ([ children for children in lstSummary  if children[1]==uniCat ])
            lstChildren.sort(key = lambda x: x[2])
            for lstRow in lstChildren:  #For each Category,print the associate User Story number, the action and the Role performing the action
                Observations.write ( ''.ljust(25)  + '{:<26}'.format(lstRow[0]) +  'Task: ' + '{:<20}'.format(lstRow[2]) +  'Task Owner: ' +  lstRow[3]+ '\n' )         
                    
    #The last line is the delimiter
    with open(Output_File,'a') as Observations:
                Observations.write('\n'+ '-' * 20+ '\n')
  

In [261]:
#Negiotiable

#User Stories are not contracts. They are placeholders for discussions. They can be updated right up to the time of Sprint Planning.

def Negiotiable (Stories, Output_File):
    #Initialize Variables:
    lstSummary=[]
    Summary=''
    User_Role=''
    User_Action=[' ']
    bflag = True

    #Logic
    for i, User_Story in enumerate(Stories):
        #print(User_Story)
        User_Story = ''.join(User_Story.split(',', maxsplit=2)[:2])
        doc = nlp(User_Story)
        bflag = True #This is set for each User Story.
        
        for tokens in doc:
            
            #Get the Role.Look at how we specifically look for certain 'types'. 
            #This is facilitated by how the User Stories are written
            if (tokens.pos_ =='NOUN' or tokens.pos_ =='PROPN') and tokens.dep_=='pobj' and  bflag==True:
                User_Role = tokens 
                bflag = False #The 'For' loop should not enter this piece of code again. Hence, using the bool variable.
                
            #The Category to which the Action is related to. 
            #Note this uses the list 'User_Action' got in next 'If' statement
            #Due to nature of user Story, this clause will never fire first. If it fires, chances are, User Story is written incorretly
            if tokens.text == str(User_Action[0]):
                Category = [child  for child in tokens.children if child.dep_=='dobj' ] #Direct Object of the Verb
                
            #The Action performed by the role
            if tokens.dep_ == 'ROOT' and tokens.text != User_Action:
                User_Action = [child  for child in tokens.children if child.dep_=='xcomp'] #The action intended, in the User Story

        Summary = [i+1, str(Category[0]).capitalize(), str(User_Action[0]).capitalize(), str(User_Role).capitalize()]
        lstSummary.append(Summary)

    #We need to get the unique Role in the User Story
    strUniqueRoles = set([Roles[3] for Roles in lstSummary])
    
    #Ready to write the observations to file
    with open(Output_File,'a') as Observations:
        Observations.write('Negotiable Criteria Observations:'+ '\n')
        Observations.write(''.ljust(20) + 'User Story #' + ''.ljust(22) + 'Task' + '\n' )         
        for uniRole in strUniqueRoles:
            Observations.write ('Role:'+ uniRole+ '\n')
            lstChildren = ([ children for children in lstSummary  if children[3]==uniRole ])
            lstChildren.sort(key = lambda x: x[2])
            for lstRow in lstChildren: 
                Observations.write (  ''.ljust(25) +  '{:<26}'.format(lstRow[0]) +   'Task: ' +  lstRow[2] + ' - ' + lstRow[1]+ '\n') 

                    
    #Last line in file
    with open(Output_File,'a') as Observations:
        Observations.write('\n'+ '-' * 20+ '\n')


In [263]:
#Valuable

#We have a data dictionary (Json) containing the 'Roles' and their 'Actions'. 
#We validate the 'Role' and 'Action' in the User Story against the 'Role' and 'Action' in the Data Dictionary. 
#If they do not match, we let the Product Owner know

def Valuable (Stories, Role_XML, Output_File):

    #Initialize Variables
    lstSummary=[]
    lstActionRoles =  []
    dictRoleData = {}
    Summary = ''
    User_Role=''
    User_Action=''
    
    #Create the dictionary that would the hold mappings between 'Role' and corresponding 'Action'
    tree = ET.parse(Role_XML) #Read XML file
    root = tree.getroot()

    #iterate the xml 
    for actions in root.findall('action'):
        
        lstActionRoles = []
        
        # iterate child elements of item 
        for role in actions.findall('role'):
            lstActionRoles.append(role.text)
    
        #Add to  dictionary
        dictRoleData[str(actions.attrib)[10:-2]] = lstActionRoles

    #Logic
    for i, User_St in enumerate(Stories):
        User_Story = ''.join(User_St.split(',', maxsplit=2)[:2])
        doc = nlp(User_Story)
        for tokens in doc:
            #The 'Role' can either be a noun or a proper noun
            if (tokens.pos_ =='NOUN' or tokens.pos_ =='PROPN') and tokens.dep_=='pobj':
                User_Role = tokens 
            if tokens.dep_ == 'ROOT':
                User_Action = [child  for child in tokens.children if child.dep_=='xcomp']
                break #A break is necessary here as further parsing of tokens can match the 'if' clauses above and give us different roles and actions

        Summary = [i+1, str(User_Role).capitalize(), str(User_Action[0]).capitalize()]
        lstSummary.append (Summary)

    #Ready to write the observations to file
    with open(Output_File,'a') as Observations:
        Observations.write('Valuable Criteria Observations:' + '\n')
        Observations.write(''.ljust(20) + 'User Story #' + ''.ljust(22) + 'Comments' + '\n' )          
   
    for lstRow in lstSummary:  
        if dictRoleData.get(str(lstRow[2])):  #Check if Action exists
            if not ( str(lstRow[1]) in dictRoleData.get(str(lstRow[2]))):  #Check if 'Role' can perform  the action.
                with open(Output_File,'a') as Observations:
                    Observations.write (''.ljust(25) + '{:<26}'.format(lstRow[0])+ 'The action \'' + lstRow[2] + '\' cannot be performed by ' + lstRow[1] + '. Please contact the Admin to update Json file if this is incorrect'+ '\n')
        else:
            with open(Output_File,'a') as Observations:
                Observations.write (''.ljust(25) +'{:<26}'.format(lstRow[0])+ 'The action \'' + lstRow[2] + '\' does not exist. Please contact Admin to update Json file.'+ '\n' )

    #Last line in file
    with open(Output_File,'a') as Observations:
                Observations.write('\n'+ '-' * 20+ '\n')

In [274]:
#Estimable

#Stories need to be such that they can be sized. Ambiguity can hurt the team's chances of sizing the story
#We try to look at the presence of conjunctions in the User Story that 'extend' what functionality the User Story needs to achieve

def Estimable (Stories, Output_File):

    #Initialize Variables
    lstSummary=[]
    storyList = []
    Summary = ''

    #Set Word combinations to check
    wordCombo =['and','or','/']

    #Logic
    for i, User_St in enumerate(Stories):

        #Split sentence into 3 parts=> 1st part deals with Roles, 2nd part deals with Requirments and 3rd part deals with Outcomes
        storyList= User_St.split(',')

        #Count occurence of "and" , "or", "/" in 1st Part. If more than 1, then multiple Roles exist. Estimation can become difficult
        andCount    = (len(re.findall('\\b' + wordCombo[0]  +'\\b', storyList[0].lower())))  
        orCount     = (len(re.findall('\\b' + wordCombo[1]  +'\\b', storyList[0].lower()))) 
        slashCount  = (len(re.findall('\\b' + wordCombo[2]  +'\\b', storyList[0].lower()))) 
        if (andCount>=1 or orCount >= 1 or slashCount>= 1) : 
            Summary = [i+1,User_St, 'Presence of multiple roles. Please have one role per User Story.']
            lstSummary.append(Summary)


        #Count occurence of "and" , "or", "/" in 'Requirements' Part. 
        #If more than 1, then multiple Requirements exist. Estimation can become difficult
        andCount    = (len(re.findall('\\b' + wordCombo[0]  +'\\b', storyList[1].lower())))
        orCount     = (len(re.findall('\\b' + wordCombo[1]  +'\\b', storyList[1].lower())))
        slashCount  = (len(re.findall('\\b' + wordCombo[2]  +'\\b', storyList[1].lower())))
        if(andCount>=1 or orCount>= 1 or slashCount>= 1) :
            Summary = [i+1,User_St, 'Presence of multiple requirements. Please have one requirement per User Story.']
            lstSummary.append(Summary)

        #Count occurence of "and" , "or", "/" in 'Outcome' Part. 
        #If more than 1, then multiple Outcomes exist. Estimation can become difficult
        andCount    = (len(re.findall('\\b' + wordCombo[0]  +'\\b', storyList[2].lower())))
        orCount     = (len(re.findall('\\b' + wordCombo[1]  +'\\b', storyList[2].lower())))
        slashCount  = (len(re.findall('\\b' + wordCombo[2]  +'\\b', storyList[2].lower())))
        if(andCount>=1 or orCount>= 1 or slashCount>= 1) :
            Summary = [i+1, User_St, 'Presence of multiple outcomes. Please have one outcome per User Story.']
            lstSummary.append(Summary)

     #Ready to write the observations to file
    with open(Output_File,'a') as Observations:
                Observations.write('Estimable Criteria Observations:'+ '\n')
            
    if lstSummary:
        for lstRow in lstSummary:
            with open(Output_File,'a') as Observations:
                Observations.write (''.ljust(20) +'User Story #:' + '{:<18}'.format(lstRow[0])+  str(lstRow[1])+ '\n')
                Observations.write(''.ljust(51) + 'Observation(s):' + lstRow[2] + '\n' )

    else:
        with open(Output_File,'a') as Observations:
            Observations.write (''.ljust(20) +'All User Stories are parsed and do not seem to have multiple functionalities to complete, in their content.' + '\n')

    #Last line in file
    with open(Output_File,'a') as Observations:
        Observations.write('\n'+ '-' * 20+ '\n')

In [277]:
#Small

#Stories need to be small enough to fit in a sprint.

def Small (Stories, Output_File):
    
    #Initialize Variables
    lstSummary=[]
    storyList = []
    Summary = ''

    #Set Word combinations to check
    wordCombo =['and','or','/']

    #Logic
    for i, User_St in enumerate(Stories):

        #Split sentence into 3 parts=> 1st part defines the Role, 2nd, the Requirement and 3rd, the Outcome
        #storyList= User_St.split(',')

        #Count word "and" , or , /  in sentence. If more than 1 then incorrect
        andCount    = (len(re.findall('\\b' + wordCombo[0]  +'\\b', User_St.lower()))) 
        orCount     = (len(re.findall('\\b' + wordCombo[1]  +'\\b', User_St.lower())))  
        slashCount  = (len(re.findall('\\b' + wordCombo[2]  +'\\b', User_St.lower()))) 

        #All 3 occurence checks
        if(andCount >=1 and orCount >=1 and slashCount >=1):
            Summary =  [i+1, User_St, 'The User Story has occurences of \''+ wordCombo[0]  + '\',\'' +  wordCombo[1]+ '\'' +  ',\'' +  wordCombo[2]+ '\'. ' + 'Reframe Story with no occurence of these. Usage of \''+ wordCombo[0]  +'\',\'' +  wordCombo[1] + '\' and ' + '"/"  increase complexity of the functionality to deliver']
            lstSummary.append(Summary)

        #2 Combinations occurence Checks
        elif (andCount >=1 and orCount >=1):
            Summary =  [i+1, User_St, 'The User Story has occurence of \''+ wordCombo[0]  + '\',\'' +  wordCombo[1]+ '\'. Reframe Story with no occurence of \'' + wordCombo[0] + '\',\'' +  wordCombo[1]+ '\'']
            lstSummary.append(Summary)
        elif (andCount >=1 and slashCount >=1)  :
            Summary =  [i+1, User_St, 'The User Story has occurence of \''+ wordCombo[0]  + '\',\''  +  wordCombo[2]+ '\'. Reframe Story with no occurence of \'' + wordCombo[0] + '\',\'' +  wordCombo[2]+ '\'']
            lstSummary.append(Summary)        
        elif (orCount >=1 and slashCount >=1)  :
            Summary =  [i+1, User_St, 'The User Story has occurence of \''+ wordCombo[1]  + '\',\''  +  wordCombo[2]+ '\'. Reframe Story with no occurence of \'' + wordCombo[1] + '\',\'' +  wordCombo[2]+ '\'']
            lstSummary.append(Summary)


        #Single occurenceheck
        elif (andCount >=1):
            Summary =  [i+1,User_St,  'The sentence has a occurence of \''+ wordCombo[0]  + '\'. Reframe Story with no occurence of \'' + wordCombo[0] + '\''  ]
            lstSummary.append(Summary)
        elif (orCount >=1):
            Summary =  [i+1,User_St, 'The sentence has a occurence of \''+ wordCombo[1]  + '\'. Reframe Story with no occurence of \'' + wordCombo[1] + '\'' ]
            lstSummary.append(Summary)
        elif (slashCount >=1):
            Summary =   [i+1,User_St, 'The sentence has a occurence of \''+ wordCombo[2]  + '\'. Reframe Story with no occurence of \'' + wordCombo[2] + '\'']
            lstSummary.append(Summary)

    #Ready to write the observations to file
    with open(Output_File,'a') as Observations:
                Observations.write('Small Criteria Observations:' + '\n')
            
    if lstSummary:
        for lstRow in lstSummary:
            with open(Output_File,'a') as Observations:
                Observations.write (''.ljust(20) +'User Story #:' + '{:<18}'.format(lstRow[0])+  str(lstRow[1])+ '\n')
                Observations.write(''.ljust(51) + 'Observation(s):' + lstRow[2] + '\n' )
    else:
         with open(Output_File,'a') as Observations:
                Observations.write (''.ljust(20) +'All User Stories are parsed and do not seem to have multiple functionalities to complete, in their content.' + '\n')

    #Last line in file
    with open(Output_File,'a') as Observations:
        Observations.write('\n'+ '-' * 20+ '\n')

In [279]:
#Testable

#Stories need to be testable

def Testable(Stories,AcceptanceCriteria, Output_File):
    
    #Initialize Variables
    lstdata =[]
    dictDatatmp={}
    #dataset=set([])
    lstTopics=[]
    lstAccCriteria =[]
    lstUserStory = []
    #dictData={}
    dictUserStory={}
    
    #Ready to write the observations to file
    with open(Output_File,'a') as Observations:
        Observations.write('Testable Criteria Observations:\n')
            
            
    for i, User_St in enumerate(Stories):
        
        #Clear these for every User Story
        lstAccCriteria = []
        lstTopics = []
        lstdata = []
        dictDatatmp ={}
        lstUserStory = []
        dictUserStory={}
        
        Acceptance = [criteria for criteria in AcceptanceCriteria if criteria[0] ==i+1]  #i starts from 0.
        
        #Append words to List
        for Acc_Criteria in Acceptance:
            doc = nlp (Acc_Criteria[1])
            lstAccCriteria.append( [ token.lemma_ for token in doc if not token.is_stop and token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])  

        #Create dict for later lookup 
        #for i,j in  enumerate(lstAccCriteria):
        #    dictdata[i]=j
        
       
        # Create Dictionary
        dicid2words = corpora.Dictionary(lstAccCriteria)

        # Create Corpus
        texts = lstAccCriteria

        # Term Document Frequency
        corpusdata = [dicid2words.doc2bow(text) for text in texts]

        # Build LDA model
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpusdata, id2word=dicid2words,num_topics=5,random_state=100,
                                               update_every=1, passes=10, alpha='auto', per_word_topics=True)

        #Select the max value Keyword in the topics
        for  topic in lda_model.print_topics(-1):
            lstdata = topic[1].split('+')

        for lsdt in lstdata:
            dictDatatmp[(lsdt.split('*')[1])]=(lsdt.split('*')[0])

        top_three = nlargest (3, dictDatatmp, key= dictDatatmp.get)
        for idx, value in enumerate(top_three):
            top_three[idx] = value.replace('"','').replace(' ','')

        lstTopics.append(top_three)
        dictDatatmp.clear()

        lstUserStory=[]
        
        #Append words from User Story to List
        doc = nlp(User_St)
        lstUserStory.append( [ token.lemma_ for token in doc if not token.is_stop])  

        #Create dict for later lookup 
        for k,story in  enumerate(lstUserStory):
            dictUserStory [k]= story

        #Print sentences having more than 2 topics
        count  = 0

        for key, value in dictUserStory.items():
            for item in lstTopics:
                if (len(set(value).intersection(set(item))))>1:
                    count +=1

            if (count < 1):
                with open(Output_File,'a') as Observations:
                    Observations.write ( '\n'+ ''.ljust(20) + 'User Story #:' + '{:<2}'.format(i+1) + User_St + '\n')
                    Observations.write (''.ljust(20) +'Top 3 topics in Acceptance Criteria for User Story #'+ '{:<2}'.format(i+1) + ': '+ str(lstTopics)+ '\n')
                    Observations.write (''.ljust(20) +'User Story #:' + '{:<2}'.format(i+1) + 'Observation: Acceptance Criteria should speak more about the User Story'+ '\n')
            count = 0
            
    #Last line in file
    with open(Output_File,'a') as Observations:
        Observations.write('\n'+ '-' * 20+ '\n')